#### data

In [1]:
import polars as pl

In [3]:
df = pl.read_csv('seattle-weather.csv')
df

date,precipitation,temp_max,temp_min,wind,weather
str,f64,f64,f64,f64,str
"""2012-01-01""",0.0,12.8,5.0,4.7,"""drizzle"""
"""2012-01-02""",10.9,10.6,2.8,4.5,"""rain"""
"""2012-01-03""",0.8,11.7,7.2,2.3,"""rain"""
"""2012-01-04""",20.3,12.2,5.6,4.7,"""rain"""
"""2012-01-05""",1.3,8.9,2.8,6.1,"""rain"""
…,…,…,…,…,…
"""2015-12-27""",8.6,4.4,1.7,2.9,"""rain"""
"""2015-12-28""",1.5,5.0,1.7,1.3,"""rain"""
"""2015-12-29""",0.0,7.2,0.6,2.6,"""fog"""


AttributeError: 'DataFrame' object has no attribute 'info'